In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import tensorflow as tf
from transformers import BertTokenizer
from transformers import *
import re


2023-08-22 23:11:11.515934: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/suyeon/.conda/envs/sy/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/home/suyeon/.conda/envs/sy/lib/python3.9/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
You are using torch==1.7.1+cu110, but torch>=1.9.0 is required to use MCTCTModel. Please upgrade torch.
You are using torch==1.7.1+cu110, but torch>=1.10.0 is required to use ViltModel. Please upgrade torch.


# 1. 데이터 로드

In [2]:
df = pd.read_csv("./Data/train_data.csv",encoding='cp949')
# 데이터 나누기

In [3]:
# k = 5
# shot_sentence = df[[f'{i+1}-Shot Sentence' for i in range(k)]]
# shot_bio = df[[f'{i+1}-Shot BIO' for i in range(k)]]

# stacked_df_sentence = shot_sentence.stack().reset_index(drop=True).to_frame('sentences')
# stacked_df_bio = shot_bio.stack().reset_index(drop=True).to_frame('bio_tag')

# final_df = pd.concat([stacked_df_sentence, stacked_df_bio], axis=1)
# final_df = final_df.drop_duplicates(subset=['sentences','bio_tag'])
# final_df.info()

In [4]:
train_ner_df, test_ner_df = train_test_split(df, test_size=450, random_state=42)

In [5]:
train_ner_df.head()

,sentences,morpheme,bio_tag,pos,format
712,먼저 성대검사와 음성검사 등을 정밀하게 하시면 원인을 아실 수 있을 겁니다 .,OO XXOOO OOOO OO OOOO OOO OOO OO O OO OOO O,O B-BD O O O O O O O O O O,"(먼저,MAG) / (성대,NNG) / (검사,NNG) / (와,JC) / (음성,...",Word | POS | Entity | BIO | Thought \n먼저 |먼저(M...
701,퇴행성 허리 디스크 진단을 받으셨군요 퇴행성만 진행된 경우에는 수술적인 치료를 하지...,XXX XX XXX OOO OOOOO OOOO OOO OOOO OOOO OOO OO...,B-DS I-DS I-DS O O O O O O O O O O,"(퇴행,NNG) / (성,XSN) / (허리,NNG) / (디스크,NNG) / (진...",Word | POS | Entity | BIO | Thought \n퇴행성 |퇴행(...
1144,특히 축구와 같은 운동 후 호흡곤란이 심해진다면 더욱 운동 유발성 천식의 가능성이 ...,OO OOO OO OO O XXXXO OOOOO OO XX XXX XXO OOOO ...,O O O O O B-ST O O B-DS I-DS I-DS O O O,"(특히,MAG) / (축구,NNG) / (와,JKB) / (같,VA) / (은,ET...",Word | POS | Entity | BIO | Thought \n특히 |특히(M...
1201,"통증이 어쩌다 한번 느껴진 경우라면 , 별 문제 없을 수 있지만 , 반복되거나 지속...",XXO OOO OO OOO OOOO O O OO OO O OOO O OOOOO OO...,B-ST O O O O O O O O O O O O O O O O O O,"(통증,NNG) / (이,JKS) / (어쩌다,MAJ) / (한,MM) / (번,N...",Word | POS | Entity | BIO | Thought \n통증이 |통증(...
332,체온상승 등으로 유발된 자율신경계의 이상반응에 의해 미주신경인 부교감신경의 과도한 ...,XXXX OOO OOO OOOOOO OOOOO OO XXXXO XXXXXO OOO ...,B-ST O O O O O B-BD B-BD O O B-ST B-ST B-DS O ...,"(체온,NNG) / (상승,NNG) / (등,NNB) / (으로,JKB) / (유발...",Word | POS | Entity | BIO | Thought \n체온상승 |체온...


In [6]:
test_ner_df.head()

,sentences,morpheme,bio_tag,pos,format
377,그래도 호전 없으면 허리 아픈지 점검 받아 보시기 바랍니다 .,OOO OO OOO XX OOO OO OO OOO OOOO O,O O O B-BD O O O O O O,"(그래도,MAJ) / (호전,NNG) / (없,VA) / (으면,EC) / (허리,...",Word | POS | Entity | BIO | Thought \n그래도 |그래도...
548,"허리 자체의 문제도 있겠지만 , 그 보다도 오히려 통증 전달 체계의 이상 여부도 확...",XX OOO OOO OOOO O O OOO OOO XX OO OOO OO OOO O...,B-BD O O O O O O O B-ST O O O O O O O,"(허리,NNG) / (자체,NNG) / (의,JKG) / (문제,NNG) / (도,...",Word | POS | Entity | BIO | Thought \n허리 |허리(N...
979,"이비인후과에서 진찰과 내시경 등으로 확인하여 제거할 수있으며 , 때로는 반복적인 검...",OOOOOOO OOO OOO OOO OOOO OOO OOOO O OOO OOOO O...,O O O O O O O O O O O O O O O O O O O O O O,"(이비인후과,NNG) / (에서,JKB) / (진찰,NNG) / (과,JC) / (...",Word | POS | Entity | BIO | Thought \n이비인후과에서 ...
1149,"질문자 님의 경우 , 목을 움직일 경우 통증이 더 유발 된다고 하면 , 목 척추 관...",OOO OO OO O OO OOO OO XXO O OO OOO OO O X XX O...,O O O O O O O B-ST O O O O O B-BD B-BD O O O O...,"(질,NNG) / (문자,NNG) / (님,XSN) / (의,JKG) / (경우,N...",Word | POS | Entity | BIO | Thought \n질문자 |질(N...
481,단순한 천식으로 의식을 잃는 경우는 없으나수술 후에 천식 발작이 오고 호흡부전이 왔...,OOO XXOO OOO OO OOO OOOOO OO XX XXO OO XXXXO O...,O B-DS O O O O O B-DS B-DS O B-ST O O O O O O O,"(단순,NNG) / (한,XSA+ETM) / (천식,NNG) / (으로,JKB) /...",Word | POS | Entity | BIO | Thought \n단순한 |단순(...


훈련 데이터와 테스트 데이터의 샘플 개수를 확인해봅시다.

In [7]:
print("학습 데이터 샘플 개수 :", len(train_ner_df))
print("테스트 데이터 샘플 개수 :", len(test_ner_df))

학습 데이터 샘플 개수 : 930
테스트 데이터 샘플 개수 : 450


In [8]:
train_data_sentence = [sent.split() for sent in train_ner_df['sentences'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['sentences'].values]
train_data_label = [tag.split() for tag in train_ner_df['bio_tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['bio_tag'].values]

In [9]:
tag_to_index = {'B-DS': 0, 'I-DS': 1, 'B-ST': 2, 'I-ST': 3, 'B-BD': 4, 'I-BD': 5, 'O': 6}
index_to_tag = {idx: tag for tag, idx in tag_to_index.items()}


In [10]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 7


# 2. 전처리 예시

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

loading file vocab.txt from cache at /home/suyeon/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/suyeon/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/tokenizer_config.json
loading configuration file config.json from cache at /home/suyeon/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 

# 3. 전처리

In [12]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)

        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [13]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 930/930 [00:00<00:00, 1260.34it/s]


In [14]:
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

  0%|          | 0/450 [00:00<?, ?it/s]

100%|██████████| 450/450 [00:00<00:00, 1400.71it/s]


# 4. 모델링과 학습

In [15]:
from transformers import TFBertForTokenClassification

In [16]:
# GPU 디바이스 설정
devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[0], 'GPU')

# 분산 전략 설정
strategy = tf.distribute.OneDeviceStrategy('GPU')

2023-08-22 23:11:27.581061: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-08-22 23:11:27.581940: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:19:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.725GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2023-08-22 23:11:27.582505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:1a:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.725GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2023-08-22 23:11:27.583024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:67:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.725GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 8

In [17]:
def compute_loss(labels, logits):

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  active_loss = tf.reshape(labels, (-1,)) != -100
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

  return loss_fn(labels, reduced_logits)

class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

def modeling(model_name, tag_size):
    model = TFBertForTokenClassification(model_name, tag_size)
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=5e-5)
    # multi-gpu 사용
  
    model.compile(optimizer=optimizer, loss=compute_loss)
    return model


In [18]:

class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):

      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list))

In [19]:
f1_score_report = F1score(X_test, y_test)

In [20]:
with strategy.scope():

  model = modeling(model_name='bert-base-uncased',
                                    tag_size = len(tag_to_index))
  f1_score_report = F1score(X_test, y_test)

model.fit(
    X_train, y_train, epochs=1, batch_size=8,
    callbacks = [f1_score_report]
    )   

# model.save_weights('./save_weight/{k}-shot_model')
model.save_weights('./save_weight/bert_multi_model_1')

loading configuration file config.json from cache at /home/suyeon/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/suyeon/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/pytorch_model.bin
Loading PyTorch w

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'Literal' and 'str'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'Literal' and 'str'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
117/117 [==============================] - 23s 89ms/step - loss: 0.5058


2023-08-22 23:11:56.381897: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_27697"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto_shard_policy = Auto

 - f1: 0.30
              precision    recall  f1-score   support

          BD       0.25      0.00      0.01       219
          DS       0.00      0.00      0.00       183
          ST       0.00      0.00      0.00       261

   micro avg       0.25      0.00      0.00       663
   macro avg       0.08      0.00      0.00       663
weighted avg       0.08      0.00      0.00       663



/home/suyeon/.conda/envs/sy/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 5. 예측

In [21]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [22]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 359.85it/s]


In [23]:
def remove_stopwords(sentence):
  with open('/home/suyeon/code/capstone2/bert_ner/stop_words.txt',"r") as f:
    stop_words = [line.rstrip('\n') for line in f]
  result = []
  sentence = sentence.split(' ')
  for i in sentence:
      result.append(i)
  return " ".join(result)

In [24]:
def ner_prediction(examples, max_seq_len, tokenizer, lang='ko'):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_list.append(index_to_tag[pred_index])

  print(pred_list)
  return " ".join(pred_list)

In [25]:
sent1 = '급성 당뇨병이 생긴거 같아요'
sent2 = '대장내시경을 통해서 우울증을 극복하자'

In [26]:
ner_prediction([sent2], max_seq_len=128, tokenizer=tokenizer)


100%|██████████| 1/1 [00:00<00:00, 683.89it/s]

['O', 'O', 'O', 'O']



2023-08-22 23:15:29.245021: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_32539"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto_shard_policy = Aut

'O O O O'

In [27]:
test_df = pd.read_csv("./Data/test_data.csv",encoding='cp949')
test_df.head()

,sentences,morpheme,bio_tag,pos,format
0,성대결절은 발성의 문제가 있을 때 생기게 됩니다 .,XXXXO OOO OOO OO O OOO OOO O,B-DS O O O O O O O,"(성대,NNG) / (결절,NNG) / (은,JX) / (발성,NNG) / (의,J...",Word | POS | Entity | BIO | Thought \n성대결절은 |성...
1,"현재 허리 통증이 심한 상태라면 , 보다 정밀한 진찰을 받아 야 합니다 .",OO XX XXO OO OOOO O OO OOO OOO OO O OOO O,O B-BD B-ST O O O O O O O O O O,"(현재,MAG) / (허리,NNG) / (통증,NNG) / (이,JKS) / (심한...",Word | POS | Entity | BIO | Thought \n현재 |현재(M...
2,시간이 가면서 이런 신호가 나온다면 감기 오고 있다는 증거가 될 수 있읍니다 .,OOO OOO OO OOO OOOO XX OO OOO OOO O O OOOO O,O O O O O B-ST O O O O O O O,"(시간,NNG) / (이,JKS) / (가,VV) / (면서,EC) / (이런,MM...",Word | POS | Entity | BIO | Thought \n시간이 |시간(...
3,"기관지 천식인지 , 기관지 과민성인지 , 기타 다른 호흡기 질환이나 심장 질환이 동...",XXX XXOO O XXX XXXOO O OO OO XXX OOOO XX OOO O...,B-DS I-DS O B-DS I-DS O O O B-BD O B-BD O O O ...,"(기관지,NNG) / (천식,NNG) / (인지,VCP+EC) / (,,SC) / ...",Word | POS | Entity | BIO | Thought \n기관지 |기관지...
4,갑상선 기능 저하증이 조절되지 않으면 간 기능 검사상 수치가 상승되는 경우가 많고 ...,XXX XX XXXO OOOO OOO O OO OOO OOO OOOO OOO OO ...,B-DS I-DS I-DS O O O O O O O O O O B-DS O O O ...,"(갑상선,NNG) / (기능,NNG) / (저하,NNG) / (증이,NNG) / (...",Word | POS | Entity | BIO | Thought \n갑상선 |갑상선...


In [28]:
test_df['predict'] = test_df['sentences'].apply(
    lambda x: ner_prediction([x], max_seq_len=128, tokenizer=tokenizer)
)

100%|██████████| 1/1 [00:00<00:00, 547.70it/s]
2023-08-22 23:15:30.668036: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_32663"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.e

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


100%|██████████| 1/1 [00:00<00:00, 398.66it/s]
2023-08-22 23:15:30.787640: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_32787"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.e

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


100%|██████████| 1/1 [00:00<00:00, 500.22it/s]
2023-08-22 23:15:30.902421: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_32911"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.e

...

100%|██████████| 1/1 [00:00<00:00, 355.00it/s]
2023-08-22 23:16:15.594071: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_88215"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.e

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


100%|██████████| 1/1 [00:00<00:00, 1056.23it/s]


['O', 'O', 'O', 'O', 'O', 'O']


2023-08-22 23:16:15.690582: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_88339"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto_shard_policy = Auto

In [29]:
test_df.head()

,sentences,morpheme,bio_tag,pos,format,predict
0,성대결절은 발성의 문제가 있을 때 생기게 됩니다 .,XXXXO OOO OOO OO O OOO OOO O,B-DS O O O O O O O,"(성대,NNG) / (결절,NNG) / (은,JX) / (발성,NNG) / (의,J...",Word | POS | Entity | BIO | Thought \n성대결절은 |성...,O O O O O O O O
1,"현재 허리 통증이 심한 상태라면 , 보다 정밀한 진찰을 받아 야 합니다 .",OO XX XXO OO OOOO O OO OOO OOO OO O OOO O,O B-BD B-ST O O O O O O O O O O,"(현재,MAG) / (허리,NNG) / (통증,NNG) / (이,JKS) / (심한...",Word | POS | Entity | BIO | Thought \n현재 |현재(M...,O O O O O O O O O O O O O
2,시간이 가면서 이런 신호가 나온다면 감기 오고 있다는 증거가 될 수 있읍니다 .,OOO OOO OO OOO OOOO XX OO OOO OOO O O OOOO O,O O O O O B-ST O O O O O O O,"(시간,NNG) / (이,JKS) / (가,VV) / (면서,EC) / (이런,MM...",Word | POS | Entity | BIO | Thought \n시간이 |시간(...,O O O O O O O O O O O O O
3,"기관지 천식인지 , 기관지 과민성인지 , 기타 다른 호흡기 질환이나 심장 질환이 동...",XXX XXOO O XXX XXXOO O OO OO XXX OOOO XX OOO O...,B-DS I-DS O B-DS I-DS O O O B-BD O B-BD O O O ...,"(기관지,NNG) / (천식,NNG) / (인지,VCP+EC) / (,,SC) / ...",Word | POS | Entity | BIO | Thought \n기관지 |기관지...,O O O O O O O O O O O O O O O O O O O O O O O O
4,갑상선 기능 저하증이 조절되지 않으면 간 기능 검사상 수치가 상승되는 경우가 많고 ...,XXX XX XXXO OOOO OOO O OO OOO OOO OOOO OOO OO ...,B-DS I-DS I-DS O O O O O O O O O O B-DS O O O ...,"(갑상선,NNG) / (기능,NNG) / (저하,NNG) / (증이,NNG) / (...",Word | POS | Entity | BIO | Thought \n갑상선 |갑상선...,O O O O O O O O O O O O O O O O O O O O O O O O


In [31]:
test_df.to_csv("./train_result/all_train/bert-base-uncased/result_1.csv",encoding='cp949')